### The Requirements:
Step 1: Choose an ETF with a minimum of 100 assets, identify those assets

Step 2: Retrieve historical data for your chosen ETF

Step 3: Calculate the price momentum factors for each asset in your ETF

Step 4: Using the price momentum factors, calculate the monthly z-factor score for each asset

Step 5: Identify long and short baskets (10 to 15 assets in each) using calculated z-factors

Step 6: Create a backtest to validate performance of your algorithm based on monthly restructuring over the previous 5 years.

Step 7: Chart:

1. Monthly portfolio return bar chart (pos/neg coloring) vs ETF

2. Monthly return for/ long picks vs short picks vs ETF

3. Cumulative portfolio return vs ETF

In [1]:
# Import Libraries
import pandas as pd
import yfinance as yf

In [5]:
# Step 1: Choose an ETF with a minimum of 100 assets, identify those assets
# etf = ["SPY"]

# Get the list of S&P 500 constituents
# SPY_tickers = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]['Symbol'].tolist()
SPY_tickers = pd.read_excel("https://www.ssga.com/us/en/intermediary/etfs/library-content/products/fund-data/etfs/us/holdings-daily-us-en-spy.xlsx", header=4).Ticker.dropna().to_list()

print(f'{len(SPY_tickers)} tickers')


504 tickers


In [8]:
# Step 2: Retrieve historical data for your chosen ETF
# Adj Close
df = yf.download(SPY_tickers, period = '10y')['Adj Close']
#df.head()

[*********************100%%**********************]  504 of 504 completed


3 Failed downloads:
['BF.B']: Exception('%ticker%: No price data found, symbol may be delisted (period=1y)')
['BRK.B', '-']: Exception('%ticker%: No data found, symbol may be delisted')


In [9]:
# Drop na
sp500 = df.dropna(how= 'all', axis= 1)
sp500

,A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2022-11-21,144.121811,13.85,147.187302,151.009521,95.709999,101.842644,57.160000,282.531189,321.489990,156.304657,...,73.760948,66.510796,107.306335,30.131836,110.914001,121.747391,113.659256,257.000000,48.387577,145.998215
2022-11-22,155.759583,13.98,149.345215,153.585449,95.279999,102.813240,57.779999,288.309418,330.880005,165.325226,...,74.981377,66.675423,110.410362,29.905355,111.498329,122.650520,114.453171,261.049988,48.816124,147.286469
2022-11-23,154.260193,14.42,150.230301,153.200974,96.629997,103.940674,58.049999,289.923767,335.779999,166.081039,...,74.773018,67.014366,109.859184,30.506021,112.161896,124.188957,117.112793,270.660004,49.063728,149.109848
2022-11-25,155.858887,14.50,147.286743,153.422043,97.669998,104.862236,58.730000,291.764496,334.299988,164.009949,...,74.346367,67.430794,109.472389,30.998371,112.409492,124.780655,118.303673,271.899994,49.282761,148.743195
2022-11-28,151.231583,13.83,143.418350,152.278244,95.300003,103.323036,58.020000,286.055206,328.970001,160.986725,...,77.590897,67.198380,106.184631,29.993979,108.556862,124.701767,116.775383,269.000000,47.835228,146.791000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-11-14,111.610001,12.25,187.440002,138.059998,126.680000,97.080002,85.500000,320.489990,604.330017,177.580002,...,86.866920,60.299999,104.290001,28.940001,101.199997,127.529999,108.070000,215.660004,35.369999,172.649994
2023-11-15,113.599998,12.42,188.009995,137.600006,128.350006,98.000000,82.660004,325.500000,595.309998,180.779999,...,87.205933,60.090000,103.660004,29.320000,100.239998,126.620003,110.500000,216.960007,36.080002,174.619995
2023-11-16,114.190002,12.19,189.710007,138.279999,126.279999,100.260002,83.709999,327.320007,602.059998,179.839996,...,85.809998,60.700001,102.459999,29.600000,101.260002,127.830002,111.550003,215.479996,35.709999,176.539993


In [ ]:

# 20 day lag time period 
starting_period = -20 -252
end_period = -20
lagged_closed_price = sp500[starting_period : end_period]

# calculating 52 Week trend 
polyfit_regression = np.polyfit((range(0,252)), lagged_closed_price, 1) 
# The first parameter of our regression is x, 0-252 days. Second is the y, Closing price for slice of data with 20 - day lag
slope_info = pd.DataFrame(polyfit_regression) # Convert so that we can use the results
slope_info.columns = sp500.columns # Setting column names to match ticker
_52_week_trend= slope_info.iloc[0] # Only need first row 
_52_week_trend

In [10]:
# Step 3: Calculate the price momentum factors for each asset in your ETF

def calculate_momentum_factors(data, lag=20):
    # Factor 1: Slope of 52-week trend line (20-day lag)
    data['Slope_52Week'] = data['Close'].pct_change(252 - lag).rolling(window=20).mean() * 100

    # Factor 2: Percent above 260-day low (20-day lag)
    data['Percent_Above_260Day_Low'] = (data['Close'] - data['Low'].rolling(window=260 - lag).min()) / (data['High'].rolling(window=260 - lag).max() - data['Low'].rolling(window=260 - lag).min()) * 100

    # Factor 3: 4/52 Week Price Oscillator (20-day lag)
    data['Price_Oscillator'] = (data['Close'].rolling(window=4).mean() / data['Close'].rolling(window=52 - lag).mean() - 1) * 100

    # Factor 4: 39-week return (20-day lag)
    data['39Week_Return'] = data['Close'].pct_change(39 - lag) * 100

    # Factor 5: 51-week Volume Price Trend (20-day lag)
    data['Volume_Price_Trend'] = (data['Close'].pct_change() * data['Volume']).rolling(window=51 - lag).sum()

    return data[['Slope_52Week', 'Percent_Above_260Day_Low', 'Price_Oscillator', '39Week_Return', 'Volume_Price_Trend']]

# Apply the function to ETF data and asset data
#etf_momentum_factors = calculate_momentum_factors(data)
all_asset = {}

for asset in SPY_tickers:
    assets_momentum_factors = data[asset].apply(calculate_momentum_factors)

# Display the calculated price momentum factors
# print("ETF Momentum Factors:")
# print(etf_momentum_factors.head())
print("\nAsset Momentum Factors:")
print(assets_momentum_factors.head())

# # Displaying the signals for each asset
# for asset, signals in assets_momentum_factors.items():
#     print(f"Signals for {asset}:\n{signals}\n")



TypeError: 'float' object is not subscriptable

In [ ]:
data

Adj Close                                                 \
                     A        AAL        AAPL        ABBV        ABNB   
Date                                                                    
2018-11-19   60.362831  35.250011   44.597614   70.844467         NaN   
2018-11-20   65.096603  35.023674   42.466839   69.743942         NaN   
2018-11-21   65.289421  35.732212   42.418835   68.208000         NaN   
2018-11-23   65.761826  37.346119   41.341442   67.796295         NaN   
2018-11-26   66.764511  37.493729   41.900547   68.279236         NaN   
...                ...        ...         ...         ...         ...   
2023-11-13  107.459999  11.780000  184.800003  138.639999  119.150002   
2023-11-14  111.610001  12.250000  187.440002  138.059998  126.680000   
2023-11-15  113.599998  12.420000  188.009995  137.600006  128.350006   
2023-11-16  114.190002  12.190000  189.710007  138.279999  126.279999   
2023-11-17  113.150002  12.290000  189.690002  138.300003  127.150002   

                                                                       ...  \
                   ABT       ACGL         ACN        ADBE         ADI  ...   
Date                                                                   ...   
2018-11-19   64.864494  28.250000  151.756714  219.690002   77.722237  ...   
2018-11-20   63.503571  27.920000  148.148605  219.729996   80.903107  ...   
2018-11-21   63.016220  28.000000  146.358490  225.979996   81.502937  ...   
2018-11-23   62.630009  28.000000  146.311890  225.559998   80.557747  ...   
2018-11-26   63.273697  28.160000  147.682404  231.960007   81.857353  ...   
...                ...        ...         ...         ...         ...  ...   
2023-11-13   95.790001  86.570000  315.630005  590.340027  170.660004  ...   
2023-11-14   97.080002  85.500000  320.489990  604.330017  177.580002  ...   
2023-11-15   98.000000  82.660004  325.500000  595.309998  180.779999  ...   
2023-11-16  100.260002  83.709999  327.320007  602.059998  179.839996  ...   
2023-11-17   99.550003  83.599998  327.829987  602.659973  183.050003  ...   

             Volume                                                         \
               WYNN      XEL       XOM     XRAY      XYL      YUM      ZBH   
Date                                                                         
2018-11-19  2223500  5332100   9430800  1997300   673600  2565300  1106529   
2018-11-20  3958700  5892000  15533700  2858400  1625100  1871100  1192019   
2018-11-21  1997600  4026500   9685300  2154400  1398900  1751900   726356   
2018-11-23   714100  1270100  10875400   697100   375900   853400   276555   
2018-11-26  2741800  4587200  13741100  2176800  1087700  1743400   620781   
...             ...      ...       ...      ...      ...      ...      ...   
2023-11-13  3779800  2960500  15308400  3955600  1261100  1230500  1702200   
2023-11-14  4061300  4250000  18260500  4303800  1500400  1353700  1909700   
2023-11-15  2544600  3439500  20137600  4925400  1297000  1309400  3648600   
2023-11-16  3022800  3970800  22469100  3552800  1314800  1554600  2300700   
2023-11-17  1678400  4327900  22984900  2413300  1286600  1089600  2991400   

                                      
              ZBRA     ZION      ZTS  
Date                                  
2018-11-19  427400  2257300  2195800  
2018-11-20  555900  2292400  2606200  
2018-11-21  328800  1676200  2271100  
2018-11-23   81000   924000   836300  
2018-11-26  200600  1664400  1908100  
...            ...      ...      ...  
2023-11-13  411200  1240300  1544900  
2023-11-14  542700  3270200  1421500  
2023-11-15  501600  2444400  1714100  
2023-11-16  275700  2217300  1566900  
2023-11-17  340600  2136200  2425100  

[1258 rows x 3018 columns]

In [ ]:
momentum_signals.items()

dict_items([('A',             signal  positions
Date                         
2018-11-19     0.0        NaN
2018-11-20     0.0        0.0
2018-11-21     0.0        0.0
2018-11-23     0.0        0.0
2018-11-26     0.0        0.0
...            ...        ...
2023-11-13     1.0        1.0
2023-11-14     1.0        0.0
2023-11-15     1.0        0.0
2023-11-16     1.0        0.0
2023-11-17     1.0        0.0

[1258 rows x 2 columns]), ('AAL',             signal  positions
Date                         
2018-11-19     0.0        NaN
2018-11-20     0.0        0.0
2018-11-21     0.0        0.0
2018-11-23     0.0        0.0
2018-11-26     0.0        0.0
...            ...        ...
2023-11-13     1.0        0.0
2023-11-14     1.0        0.0
2023-11-15     1.0        0.0
2023-11-16     1.0        0.0
2023-11-17     1.0        0.0

[1258 rows x 2 columns]), ('AAPL',             signal  positions
Date                         
2018-11-19     0.0        NaN
2018-11-20     0.0        0.0
2018-11-21 